# PostgreSQL Data Wrangling

# *Import Necessary Library*

In [1]:
import json
import psycopg2
from psycopg2 import extras

# *Define Database Parameter*

In [2]:
DB_NAME="sales_db"
DB_USER="postgres"
DB_PASSWORD="duckdb"
DB_HOST="0.0.0.0"
DB_PORT="5432"

# *Create Postgresql Connection*

In [118]:
try:
    # Connect to your PostgreSQL database
    conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
    cur = conn.cursor()
    print("Database connection successful")
except (psycopg2.OperationalError, psycopg2.DatabaseError, FileNotFoundError) as e:
        print(f"An error occurred: {e}")

Database connection successful


# *SQL statement to insert a new record*

In [4]:
insert_sql = """
            INSERT INTO records (
                order_id, item_name, quantity, price_per_unit, total_price, order_date, region, payment_method, notes,
                customer_id, customer_email, customer_age, customer_street, customer_city, customer_zip, status
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            );
        """

# *Open and load the JSON data from the file*

In [5]:
with open('sales_record.json', 'r') as f:
            data = json.load(f)

# *Iterate over each record in the JSON data* 

In [6]:
for record in data:
    # Extract customer information
    customer_info = record.get('customer_info', {})
    customer_address = customer_info.get('address', {})

    # Handle the 'total_price' field which can be a string with a '$'
    total_price = record.get('total_price')
    if isinstance(total_price, str) and '$' in total_price:
        total_price = float(total_price.replace('$', ''))

    # Prepare the data for insertion.
    # You must ensure the order and types match your SQL statement and table.
    record_values = (
        record.get('order_id'),
        record.get('item_name'),
        record.get('quantity'),
        record.get('price_per_unit'),
        total_price,
        record.get('order_date'),
        record.get('region'),
        record.get('payment_method'),
        record.get('notes'),
        customer_info.get('customer_id'),
        customer_info.get('email'),
        customer_info.get('age'),
        customer_address.get('street'),
        customer_address.get('city'),
        customer_address.get('zip'),
        record.get('status')
        )
    # Insert the record into the database
    cur.execute(insert_sql, record_values)

# Commit the changes and close the connection
conn.commit()
print("Data imported successfully!")

Data imported successfully!


# *Checking importing result*

In [7]:
my_query = ''' SELECT * from records limit(5) '''

cur.execute(my_query)
data = cur.fetchall()
print(data[0])

('ORD-0001', 'Headphones', 4, Decimal('1015.74'), Decimal('4062.96'), datetime.date(2024, 9, 3), 'West', 'Bank Transfer', 'CUST-3916', 'johnny89@example.org', None, '537 Main St', 'Harrelltown', 65592, 'Pending', None)


# *--- Data Observation ----*

In [8]:
import pandas as pd

columns = [desc[0] for desc in cur.description]
df = pd.DataFrame(data, columns = columns)
df.head()

,order_id,item_name,quantity,price_per_unit,total_price,order_date,region,payment_method,customer_id,customer_email,customer_age,customer_street,customer_city,customer_zip,status,notes
0,ORD-0001,Headphones,4,1015.74,4062.96,2024-09-03,West,Bank Transfer,CUST-3916,johnny89@example.org,NaN,537 Main St,Harrelltown,65592,Pending,None
1,ORD-0002,SSD,10,952.47,9524.70,2024-12-12,South,Credit Card,CUST-6310,brenda92@example.com,32.0,152 Main St,Rebeccahaven,67092,Pending,Special handling required
2,ORD-0003,Monitor,5,678.01,3390.05,2024-10-25,West,Bank Transfer,CUST-1522,heatherbarnett@example.org,23.0,77 Main St,Port Yolandamouth,60182,Refunded,None
3,ORD-0004,Webcam,10,365.39,3653.90,2025-05-01,South,Cash,CUST-8180,lorraineprice@example.net,29.0,509 Main St,Gailmouth,59580,Cancelled,None
4,ORD-0005,SSD,3,249.20,747.60,2024-12-12,West,Crypto,CUST-8477,michaeljohnson@example.org,55.0,154 Main St,East Tinaside,45643,Pending,None


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         5 non-null      object 
 1   item_name        5 non-null      object 
 2   quantity         5 non-null      int64  
 3   price_per_unit   5 non-null      object 
 4   total_price      5 non-null      object 
 5   order_date       5 non-null      object 
 6   region           5 non-null      object 
 7   payment_method   5 non-null      object 
 8   customer_id      5 non-null      object 
 9   customer_email   5 non-null      object 
 10  customer_age     4 non-null      float64
 11  customer_street  5 non-null      object 
 12  customer_city    5 non-null      object 
 13  customer_zip     5 non-null      int64  
 14  status           5 non-null      object 
 15  notes            1 non-null      object 
dtypes: float64(1), int64(2), object(13)
memory usage: 772.0+ bytes


# *Checking duplicate value*

In [10]:
duplicate_order_id = '''
    SELECT  order_id, COUNT(order_id) 
    FROM sales_records
    GROUP BY order_id
    HAVING COUNT(order_id) > 1;
'''

In [11]:
cur.execute(duplicate_order_id)
duplicates = cur.fetchall()
if duplicates:
    print("Duplicate order_id found")
else:
    print("No duplicate found on order_id")

No duplicate found on order_id


# *Column value not Null*

In [75]:
def check_for_none(cursor, table_name, column_name):
    query = f"SELECT * FROM {table_name} WHERE {column_name} IS NULL;"
    print(f"Executing {query}")
    cursor.execute(query)
    result = cursor.fetchall()
    if result:
        print(f"Found NULL on {column_name}")
    else:
        print(f"Not found NULL on {column_name}")

In [13]:
ages = check_for_none(cur, "records", "customer_age")
ages

Executing SELECT * FROM records WHERE customer_age IS NULL;
Found NULL on customer_age


In [14]:
notes = check_for_none(cur, "records", "notes")
notes

Executing SELECT * FROM records WHERE notes IS NULL;
Found NULL on notes


In [15]:
#columns

In [79]:
def null_check(cursor=cur, table_name="records", column_name=None):
    query = f"SELECT * FROM {table_name} WHERE {column_name} IS NULL;"
    cursor.execute(query)
    result = cursor.fetchall()
    if result:        
        return "Found"
    else:
        return "Not Found"

In [17]:
results = [null_check(column_name=col) for col in columns]
#results

# *Create dictionary to see result*

In [18]:
dict_null = {col:[val] for col,val in zip(columns, results)}
dict_null

{'order_id': ['Not Found'],
 'item_name': ['Not Found'],
 'quantity': ['Not Found'],
 'price_per_unit': ['Not Found'],
 'total_price': ['Not Found'],
 'order_date': ['Not Found'],
 'region': ['Not Found'],
 'payment_method': ['Found'],
 'customer_id': ['Not Found'],
 'customer_email': ['Not Found'],
 'customer_age': ['Found'],
 'customer_street': ['Not Found'],
 'customer_city': ['Not Found'],
 'customer_zip': ['Not Found'],
 'status': ['Not Found'],
 'notes': ['Found']}

In [19]:
null_check_df = pd.DataFrame(dict_null)
null_check_df

,order_id,item_name,quantity,price_per_unit,total_price,order_date,region,payment_method,customer_id,customer_email,customer_age,customer_street,customer_city,customer_zip,status,notes
0,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Found,Not Found,Not Found,Found,Not Found,Not Found,Not Found,Not Found,Found


In [33]:
payment_null = '''SELECT payment_method, COUNT(payment_method) FROM records
                GROUP BY payment_method ORDER BY COUNT(payment_method) DESC;
                '''

cur.execute(payment_null)
payment_null_result = cur.fetchall()
payment_null_result

[('Bank Transfer', 222),
 ('Crypto', 208),
 ('Credit Card', 205),
 ('PayPal', 178),
 ('Cash', 167),
 (None, 0)]

In [39]:
notes_null = '''SELECT notes, COUNT(notes) FROM records 
            GROUP BY notes ORDER BY COUNT(notes) DESC;
            '''

cur.execute(notes_null)
notes_null_result = cur.fetchall()
notes_null_result

[('Special handling required', 87), (None, 0)]

In [44]:
age_null = '''SELECT COUNT(customer_age) FROM records
            WHERE customer_age IS NOT NULL;
            '''

cur.execute(age_null)
age_null_result = cur.fetchall()
age_null_result

[(892,)]

# *Data Cleaning*

# To fillin null age with median

In [54]:
median_age = ''' UPDATE records SET customer_age =(
                SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY customer_age)
                FROM records)
                WHERE customer_age IS NULL;
            '''
cur.execute(median_age)
# Commit the changes and close the connection
conn.commit()

# customer_age null value to check

In [60]:
customer_age = null_check(column_name="customer_age")
customer_age

'Not Found'

# Update null payment_method with Cash because the number is small

In [72]:
payment_method = ''' UPDATE records SET payment_method = 'Cash'
                    WHERE payment_method IS NULL;
                '''
cur.execute(payment_method)
# Commit the changes and close the connection
conn.commit()

In [80]:
payment_method = null_check(column_name="payment_method")
payment_method

'Not Found'

# Update notes column rename to special_handling 
# with boolean data type

In [81]:
notes_query =''' UPDATE records SET notes = true where notes IS NOT NULL;
            '''
cur.execute(notes_query)
# Commit the changes and close the connection
conn.commit()

In [82]:
notes2_query =''' UPDATE records SET notes = false where notes IS NULL;
            '''
cur.execute(notes2_query)
# Commit the changes and close the connection
conn.commit()

In [87]:
special_handling = ''' ALTER TABLE records RENAME COLUMN notes TO special_handling;
                '''
cur.execute(special_handling)
# Commit the changes and close the connection
conn.commit()

# Checking notes to special_handling

In [88]:
sp_handling = ''' SELECT special_handling, count(special_handling) FROM records
                GROUP BY special_handling ORDER BY count(special_handling) DESC;
            '''
cur.execute(sp_handling)
sp_handling_result = cur.fetchall()
sp_handling_result

[('false', 913), ('true', 87)]

# *Generate Business Insight*

# 1. Best Selling Item

In [89]:
best_selling_item = ''' SELECT item_name, sum(total_price) FROM records
                        GROUP BY item_name ORDER BY sum(total_price) DESC LIMIT 5;
                    '''
cur.execute(best_selling_item)
best_selling_item_result = cur.fetchall()
best_selling_item_result

[('SSD', Decimal('524134.23')),
 ('CPU', Decimal('483523.42')),
 ('Webcam', Decimal('460301.27')),
 ('Keyboard', Decimal('436591.17')),
 ('Mouse', Decimal('407695.41'))]

In [108]:
df_best_selling_item = pd.DataFrame(best_selling_item_result, columns=[["item_sale","total_price"]])
df_best_selling_item

,item_sale,total_price
0,SSD,524134.23
1,CPU,483523.42
2,Webcam,460301.27
3,Keyboard,436591.17
4,Mouse,407695.41


# 2. Sales Distribution By Age

In [105]:
sale_distribution_age = '''
    SELECT
        CASE
            WHEN customer_age IS NULL THEN 'Other/Unknown'
            WHEN (customer_age)::INT >= 0 AND (customer_age)::INT < 18 THEN '0-17'
            WHEN (customer_age)::INT >= 18 AND (customer_age)::INT < 25 THEN '18-24'
            WHEN (customer_age)::INT >= 25 AND (customer_age)::INT < 35 THEN '25-34'
            WHEN (customer_age)::INT >= 35 AND (customer_age)::INT < 50 THEN '35-49'
            WHEN (customer_age)::INT >= 50 AND (customer_age)::INT < 65 THEN '50-64'
            WHEN (customer_age)::INT >= 65 AND (customer_age)::INT < 100 THEN '65-99'
            ELSE 'Other/Unknown'
        END AS age_group,
        COUNT(*) AS total_price
    FROM
        records
    GROUP BY
        age_group
    ORDER BY
        age_group;
    '''
cur.execute(sale_distribution_age)
sale_distribution_age_result = cur.fetchall()
sale_distribution_age_result

[('18-24', 107),
 ('25-34', 166),
 ('35-49', 377),
 ('50-64', 247),
 ('65-99', 103)]

In [109]:
df_sale_distribution_age = pd.DataFrame(sale_distribution_age_result, columns=[["age_group","total_price"]])
df_sale_distribution_age

,age_group,total_price
0,18-24,107
1,25-34,166
2,35-49,377
3,50-64,247
4,65-99,103


# 3. Sales Distribution By Region

In [113]:
sales_by_region = ''' SELECT region, sum(total_price) FROM records
                    GROUP BY region ORDER BY sum(total_price) DESC;
                '''
cur.execute(sales_by_region)
sales_by_region_result = cur.fetchall()
sales_by_region_result

[('Central', Decimal('931008.98')),
 ('South', Decimal('900212.94')),
 ('North', Decimal('831611.66')),
 ('West', Decimal('797602.12')),
 ('East', Decimal('785278.18'))]

In [115]:
df_sales_by_region_result = pd.DataFrame(sales_by_region_result, columns=[["region","total_price"]])
df_sales_by_region_result

,region,total_price
0,Central,931008.98
1,South,900212.94
2,North,831611.66
3,West,797602.12
4,East,785278.18


# 4. Status Transaction Breakdown

In [119]:
status_query = ''' SELECT status, SUM(total_price) FROM records
                GROUP BY status ORDER BY SUM(total_price) DESC;
            '''
cur.execute(status_query)
status_query_result = cur.fetchall()
status_query_result

[('Pending', Decimal('1200341.40')),
 ('Completed', Decimal('1072176.02')),
 ('Cancelled', Decimal('998244.37')),
 ('Refunded', Decimal('974952.09'))]

In [120]:
df_status_query_result = pd.DataFrame(status_query_result, columns=[["status","total_price"]])
df_status_query_result

,status,total_price
0,Pending,1200341.40
1,Completed,1072176.02
2,Cancelled,998244.37
3,Refunded,974952.09


# 5. Payment Method To Total Sale

In [121]:
payment_query = ''' SELECT payment_method, SUM(total_price) FROM records
                GROUP BY payment_method ORDER BY SUM(total_price) DESC;
            '''
cur.execute(payment_query)
payment_query_result = cur.fetchall()
payment_query_result

[('Bank Transfer', Decimal('929087.61')),
 ('Cash', Decimal('857847.60')),
 ('Crypto', Decimal('845416.68')),
 ('PayPal', Decimal('817803.41')),
 ('Credit Card', Decimal('795558.58'))]

In [122]:
df_payment_query_result = pd.DataFrame(payment_query_result, columns=[["payment_method","total_price"]])
df_payment_query_result

,payment_method,total_price
0,Bank Transfer,929087.61
1,Cash,857847.60
2,Crypto,845416.68
3,PayPal,817803.41
4,Credit Card,795558.58


In [123]:
cur.close()

In [124]:
conn.close()